# **Preprocesamiento y análisis de datos multivariados**

### **US financial market**

*Marcela Ibarra Mora*

*A01231973*

In [6]:
#Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [7]:
#Load data
data = pd.read_csv('us2022q2a.csv')
name_columns = ["firm","Name","N","Class","Country of Origin","Type of Asset","Sector NAICS level 1","Exchange / Src","Sector Economatica","Sector NAICS last available","partind"]
df_firms = pd.read_csv('usfirms2022.csv', names = name_columns)
df_firms = df_firms.drop(["N","Class","Country of Origin","Type of Asset","Exchange / Src","Sector Economatica","Sector NAICS last available","partind"], axis=1)

#merge both data frames
data = data.merge(df_firms, how='left', on='firm')

In [8]:
#defining varibles
data ["Market Value"] = data["originalprice"]*data["sharesoutstanding"]
data["Book Value"] = data["totalassets"]-data["totalliabilities"]
data["Ebit"] = data["revenue"] - data["cogs"] - data["sgae"]
data["originalprice"] = data["originalprice"].replace(0,np.nan)
data["totalassets"] = data["totalassets"].replace(0,np.nan)
data["OPM_sum"] = data["Ebit"] + data["revenue"]
data["Op_profitmargin"] = data["OPM_sum"]/data["revenue"]  
data["Op_profitmargin"] = data["Op_profitmargin"].replace([np.inf, -np.inf], np.nan)  
data["net_income"] = data["Ebit"] - data["otheropexp"] - data["incometax"] - data["finexp"] + data["extraincome"]
data["Profit margin"] = (data["net_income"] / data["revenue"])  #Porcentaje
data["Profit margin"] = data["Profit margin"].replace([np.inf, -np.inf], np.nan)


In [9]:
#Get stock returns (annual)
data["r"] = np.log(data["adjprice"]) - np.log(data["adjprice"].shift(4))
data["F1_ret"] = data["r"].shift(-1)

In [10]:
#Earning per share:
data["EPSP"] = (data['net_income'] / data["sharesoutstanding"])/data["originalprice"]
#Book to Market Ratio
data["SAG"] = (data["revenue"] / (data["revenue"].shift(-4)))-1
data["SAG"] = data["SAG"].replace([np.inf, -np.inf], 0)
#Book to market ratio
data["B_MRatio"] = (data["totalassets"] - data["totalliabilities"]) / (data["originalprice"] * data["sharesoutstanding"])
#Short financial leverage
data["SFL"] = data["shortdebt"] / data["totalassets"]
#Long financial leverage
data["LFL"] = data["longdebt"] / data["totalassets"]
#To classify by size
data.sort_values("Market Value").groupby("q")
data["size"] = pd.cut(data["Market Value"], bins=3, labels = ["small","medium","big"])
d = pd.get_dummies(data["size"],drop_first=-1)
data[["Medium","Big"]] = d[["medium","big"]]
data = data.replace(np.nan, 0)


### **Variance - Covariance Matrix**

For this analysis, we'll only consider the second to last quarter for each firm

The variance - covariance matrix is a square matrix used to obtain the variance and covariance between multiple variables, and it's used as an estimator of parameters in a statistical model and to calculate standard errors. 

In [11]:
df_mask = data['q'] == '2022q1'
dataSTL = data[df_mask]
dataSTL = dataSTL.reset_index(drop=True)

In [12]:
x = dataSTL[["B_MRatio","Op_profitmargin","SFL","EPSP","Medium","Big"]].reset_index(drop=True)
x

,B_MRatio,Op_profitmargin,SFL,EPSP,Medium,Big
0,0.129778,1.224612,0.000000,0.007126,0,0
1,0.374854,1.273003,0.000063,0.028098,0,0
2,1.794935,1.436482,0.000000,-0.022229,0,0
3,-0.754610,0.806383,0.035341,-0.138008,0,0
4,1.982019,1.073554,0.000000,0.044556,0,0
...,...,...,...,...,...,...
3602,0.307988,0.772467,0.005220,-0.035760,0,0
3603,0.403424,0.882660,0.000000,-0.266288,0,0
3604,0.043502,1.187813,0.005006,0.006782,0,0
3605,1.154503,-12.305364,0.093572,-0.241984,0,0


In [14]:
x = x.to_numpy()
print(x.shape)
x

(3607, 6)


array([[ 1.29777902e-01,  1.22461171e+00,  0.00000000e+00,
         7.12594998e-03,  0.00000000e+00,  0.00000000e+00],
       [ 3.74853934e-01,  1.27300334e+00,  6.25469102e-05,
         2.80975699e-02,  0.00000000e+00,  0.00000000e+00],
       [ 1.79493539e+00,  1.43648170e+00,  0.00000000e+00,
        -2.22291939e-02,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 4.35017059e-02,  1.18781302e+00,  5.00625782e-03,
         6.78240330e-03,  0.00000000e+00,  0.00000000e+00],
       [ 1.15450305e+00, -1.23053642e+01,  9.35717070e-02,
        -2.41984462e-01,  0.00000000e+00,  0.00000000e+00],
       [ 7.48904237e-01,  0.00000000e+00,  2.83018402e-03,
        -9.75748982e-02,  0.00000000e+00,  0.00000000e+00]])

In [15]:
#calculate the variance - covariance Matrix
ones = np.ones([x.shape[0],1])
a = np.dot(x.transpose(),ones)
b = np.dot(a,a.transpose()) / x.shape[0]
xp_x = np.dot(x.transpose(),x)
c = xp_x - b
d = 1 / (x.shape[0] - 1)
Var_CovMat = d*c
pd.DataFrame(Var_CovMat)

,0,1,2,3,4,5
0,0.351506,1.879303,-0.005491,-0.002620,-3.679270e-04,-2.654576e-04
1,1.879303,21902.737486,-1.317390,0.495691,7.437373e-03,5.061763e-03
2,-0.005491,-1.317390,0.011594,-0.000923,-1.832809e-05,-2.281999e-06
3,-0.002620,0.495691,-0.000923,0.014561,2.099448e-05,1.666205e-05
4,-0.000368,0.007437,-0.000018,0.000021,8.312548e-04,-4.612957e-07
5,-0.000265,0.005062,-0.000002,0.000017,-4.612957e-07,5.543236e-04


In [16]:
# Using the function .cov to check if the last result is correct
x = dataSTL[["B_MRatio","Op_profitmargin","SFL","EPSP","Medium","Big"]]
x.cov()

,B_MRatio,Op_profitmargin,SFL,EPSP,Medium,Big
B_MRatio,0.351506,1.879303,-0.005491,-0.002620,-3.679270e-04,-2.654576e-04
Op_profitmargin,1.879303,21902.737486,-1.317390,0.495691,7.437373e-03,5.061763e-03
SFL,-0.005491,-1.317390,0.011594,-0.000923,-1.832809e-05,-2.281999e-06
EPSP,-0.002620,0.495691,-0.000923,0.014561,2.099448e-05,1.666205e-05
Medium,-0.000368,0.007437,-0.000018,0.000021,8.312548e-04,-4.612957e-07
Big,-0.000265,0.005062,-0.000002,0.000017,-4.612957e-07,5.543236e-04


### **Leverage**

Points with high leverage, are points that follow the tendency but have an extreme value. These points, depending on the context, can affect the models and their predictions

We can find these leverage points with the hat matrix using the formula $ H=X(X'X)^{-1} $. This matrix helps us obtain the leverages when the value of x has a extreme value. 

In [17]:
# Getting the variables to calculate the leverage points
x = dataSTL[["B_MRatio","Op_profitmargin","SFL","EPSP","Medium","Big"]].reset_index(drop=True)
ones = np.ones(x.shape[0])
x.insert(0,'ones',ones, True)
x_t = x.transpose()

In [18]:
# calculating the hat matrix
w =np.dot(x_t,x)
a = np.linalg.inv(w)
b = np.dot(x,a)
hat = np.dot(b,x_t)
pd.DataFrame(hat)

,0,1,2,3,4,5,6,7,8,9,...,3597,3598,3599,3600,3601,3602,3603,3604,3605,3606
0,0.000445,0.000375,-0.000106,0.000637,-0.000137,-0.000458,0.000433,0.000484,-0.000010,0.000316,...,0.000423,0.000471,0.000388,0.000245,0.000422,0.000364,0.000235,0.000469,-0.000076,0.000196
1,0.000375,0.000363,0.000142,0.000331,0.000178,-0.000138,0.000370,0.000385,-0.000011,0.000323,...,0.000341,0.000374,0.000315,0.000301,0.000336,0.000311,0.000099,0.000382,-0.000040,0.000203
2,-0.000106,0.000142,0.001552,-0.001022,0.001750,0.002092,-0.000072,-0.000237,-0.000028,0.000289,...,-0.000092,-0.000224,-0.000207,0.000368,-0.000158,0.000063,0.000123,-0.000194,0.000855,0.000495
3,0.000637,0.000331,-0.001022,0.001890,-0.001379,-0.001366,0.000604,0.000789,0.000011,0.000232,...,0.000701,0.000776,0.000666,-0.000012,0.000725,0.000553,0.001019,0.000725,0.000125,0.000247
4,-0.000137,0.000178,0.001750,-0.001379,0.002065,0.002225,-0.000099,-0.000298,-0.000027,0.000318,...,-0.000166,-0.000285,-0.000234,0.000484,-0.000224,0.000006,-0.000223,-0.000238,0.000678,0.000426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3602,0.000364,0.000311,0.000063,0.000553,0.000006,-0.000084,0.000358,0.000390,-0.000011,0.000291,...,0.000372,0.000383,0.000310,0.000206,0.000359,0.000342,0.000406,0.000377,0.000191,0.000282
3603,0.000235,0.000099,0.000123,0.001019,-0.000223,0.000551,0.000235,0.000289,-0.000025,0.000187,...,0.000400,0.000291,0.000136,-0.000128,0.000328,0.000406,0.001495,0.000244,0.001155,0.000636
3604,0.000469,0.000382,-0.000194,0.000725,-0.000238,-0.000573,0.000455,0.000516,-0.000006,0.000313,...,0.000445,0.000503,0.000426,0.000242,0.000451,0.000377,0.000244,0.000498,-0.000110,0.000180
3605,-0.000076,-0.000040,0.000855,0.000125,0.000678,0.001867,-0.000055,-0.000111,0.000016,0.000127,...,0.000075,-0.000080,0.000027,0.000094,0.000049,0.000191,0.001155,-0.000110,0.001557,0.000650


In [19]:
# verifing that the sum of the diagonal of the hat matrix is equal to the # of columns
# This diagonal is where we can find the point with the highest leverages
hat.diagonal().sum()

7.0

In [20]:
# getting the hat matrix diagonal
hat_d = hat.diagonal()
pd.DataFrame(hat_d)

,0
0,0.000445
1,0.000363
2,0.001552
3,0.001890
4,0.002065
...,...
3602,0.000342
3603,0.001495
3604,0.000498
3605,0.001557


In [21]:
# mean of the hat matrix diagonal
hat_mean = (hat_d.sum())/hat.shape[0]
hat_mean

0.0019406709176601053

In [22]:
#Getting the high leverage points
mask = hat_d > (3*hat_mean)
high_leverage = hat_d[mask]
high_leverage

array([0.50001108, 0.04629149, 0.01564205, 0.00796085, 0.02016852,
       0.01118322, 0.006675  , 0.01118763, 0.00750478, 0.02207944,
       0.00679319, 0.3333358 , 0.00732216, 0.0077295 , 0.04998853,
       0.01628761, 0.01389353, 0.00696007, 0.00937006, 0.00805004,
       0.21223925, 0.03493159, 0.01295984, 0.0078905 , 0.02217546,
       0.01101327, 0.22059799, 0.00732819, 0.01123522, 0.01036875,
       0.01005652, 0.27075579, 0.00705369, 0.06557237, 0.0076362 ,
       0.00652244, 0.00923787, 0.05234085, 0.04389426, 0.33333766,
       0.00986641, 0.00817756, 0.00923692, 0.00756448, 0.00975682,
       0.59694379, 0.03399465, 0.00703736, 0.01263821, 0.04365969,
       0.0075326 , 0.01158709, 0.50001108, 0.00709522, 0.0059081 ,
       0.01508059, 0.18601671, 0.02720846, 0.01205781, 0.01668877,
       0.00838278, 0.00633862, 0.01191185, 0.02340595, 0.00867054,
       0.03308635, 0.0375898 , 0.00684746, 0.00960248, 0.00626869,
       0.00861815, 0.09058254, 0.00918054, 0.33334166, 0.01304

In [23]:
# function to verify the previus result
import statsmodels.formula.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence as olsi

y = dataSTL[["F1_ret"]].reset_index(drop=True)
x.insert(7,'F1_ret',y, True)
x

result = sm.ols(formula="F1_ret ~ B_MRatio + Op_profitmargin + SFL + EPSP + Medium + Big", data=x).fit()

leverage_pts = olsi(result).hat_matrix_diag

mask = leverage_pts > (3*hat_mean)
high_leverage1 = leverage_pts[mask]
high_leverage1

array([0.50001108, 0.04629149, 0.01564205, 0.00796085, 0.02016852,
       0.01118322, 0.006675  , 0.01118763, 0.00750478, 0.02207944,
       0.00679319, 0.3333358 , 0.00732216, 0.0077295 , 0.04998853,
       0.01628761, 0.01389353, 0.00696007, 0.00937006, 0.00805004,
       0.21223925, 0.03493159, 0.01295984, 0.0078905 , 0.02217546,
       0.01101327, 0.22059799, 0.00732819, 0.01123522, 0.01036875,
       0.01005652, 0.27075579, 0.00705369, 0.06557237, 0.0076362 ,
       0.00652244, 0.00923787, 0.05234085, 0.04389426, 0.33333766,
       0.00986641, 0.00817756, 0.00923692, 0.00756448, 0.00975682,
       0.59694379, 0.03399465, 0.00703736, 0.01263821, 0.04365969,
       0.0075326 , 0.01158709, 0.50001108, 0.00709522, 0.0059081 ,
       0.01508059, 0.18601671, 0.02720846, 0.01205781, 0.01668877,
       0.00838278, 0.00633862, 0.01191185, 0.02340595, 0.00867054,
       0.03308635, 0.0375898 , 0.00684746, 0.00960248, 0.00626869,
       0.00861815, 0.09058254, 0.00918054, 0.33334166, 0.01304

In [24]:
# getting the index of the high leverage points
leverage_index = np.nonzero(hat_d > (3 * hat_mean))
leverage_index

(array([   8,   41,   57,   68,   81,   91,   93,  107,  146,  147,  149,
         201,  287,  340,  368,  388,  390,  437,  588,  607,  631,  722,
         752,  765,  767,  800,  905,  907,  937,  992, 1030, 1138, 1236,
        1262, 1266, 1318, 1328, 1361, 1396, 1403, 1471, 1510, 1528, 1560,
        1564, 1595, 1599, 1633, 1723, 1856, 1979, 2059, 2132, 2134, 2191,
        2223, 2364, 2432, 2444, 2465, 2485, 2654, 2658, 2698, 2740, 2826,
        2928, 2957, 3038, 3121, 3149, 3181, 3220, 3225, 3243, 3256, 3372,
        3386, 3410, 3414, 3483, 3492, 3529, 3577], dtype=int64),)

---
### **Outliers**

Outliers are points where the data does not follow the trend. We use the Residuals and the standardized residuals to obtain these points, as well as the real value for y and the prediction. Using the formulas:
$$
ei = y - \hat{y}
$$
$$
ri = \frac{ei}{\sqrt{MSE(1-h_{ii})}}
$$

In [25]:
# Getting the y and the prediction
y = dataSTL[["F1_ret"]].reset_index(drop=True)
y = pd.DataFrame(y)
y['pred'] = np.dot(hat,y['F1_ret']) #y^ = Hy
y

,F1_ret,pred
0,-0.213296,-0.333568
1,0.217886,-0.301955
2,-0.222528,-0.548925
3,-0.514447,-0.632480
4,-0.475675,-0.397707
...,...,...
3602,0.000000,-0.457799
3603,-0.982014,-1.041509
3604,0.093173,-0.328616
3605,-3.482115,-1.075339


In [26]:
# Calculating the mean square error of the prediction 
mse = (y['F1_ret']-y['pred'])**2
mse = mse.sum()/y.shape[0]
mse

0.36533214767257327

In [27]:
# ei : ordinary residuals 
y['ei'] = y['F1_ret'] - y['pred']

# H_ii : hat matrix diagonal (leverages)
y['H_ii'] = hat_d

# ri : Standardized residuals
y['ri'] = y['ei'] / np.sqrt(mse*(1-y['H_ii']))

# ri_abs : Absolute Standardized residuals
y['ri_abs'] = y['ri'].abs()

y

,F1_ret,pred,ei,H_ii,ri,ri_abs
0,-0.213296,-0.333568,0.120272,0.000445,0.199030,0.199030
1,0.217886,-0.301955,0.519841,0.000363,0.860211,0.860211
2,-0.222528,-0.548925,0.326397,0.001552,0.540431,0.540431
3,-0.514447,-0.632480,0.118034,0.001890,0.195467,0.195467
4,-0.475675,-0.397707,-0.077968,0.002065,-0.129128,0.129128
...,...,...,...,...,...,...
3602,0.000000,-0.457799,0.457799,0.000342,0.757539,0.757539
3603,-0.982014,-1.041509,0.059495,0.001495,0.098506,0.098506
3604,0.093173,-0.328616,0.421789,0.000498,0.698006,0.698006
3605,-3.482115,-1.075339,-2.406776,0.001557,-3.985016,3.985016


In [28]:
# Getting Outliers
outliers = y[y['ri_abs'] > 3]
outliers

,F1_ret,pred,ei,H_ii,ri,ri_abs
68,-2.961467,-0.960512,-2.000955,0.007961,-3.323755,3.323755
80,-2.869335,-0.340539,-2.528795,0.000537,-4.184912,4.184912
93,-3.821337,-1.525371,-2.295966,0.006675,-3.811324,3.811324
163,-2.313630,-0.407863,-1.905767,0.000450,-3.153724,3.153724
197,-2.754820,-0.752175,-2.002645,0.000589,-3.314271,3.314271
347,-2.485650,-0.387283,-2.098368,0.000307,-3.472196,3.472196
368,-1.982500,-4.571925,2.589425,0.049989,4.395365,4.395365
390,-0.666268,-2.473659,1.807390,0.013894,3.011245,3.011245
399,-2.699358,-0.754655,-1.944703,0.000690,-3.218542,3.218542
500,1.586539,-0.244480,1.831019,0.000574,3.030216,3.030216


In [29]:
# Getting the index of those Outliers
outliers_index = outliers.index
outliers_index

Int64Index([  68,   80,   93,  163,  197,  347,  368,  390,  399,  500,  531,
             631,  712,  808,  869,  908,  963, 1190, 1236, 1262, 1310, 1318,
            1361, 1390, 1398, 1424, 1580, 1581, 1599, 1791, 2301, 2417, 2613,
            2617, 2723, 2781, 2826, 2875, 2922, 2997, 3032, 3123, 3182, 3202,
            3605],
           dtype='int64')

### **Multicollinearity**

In [30]:
#Import vif function
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculating vif
x = dataSTL[["B_MRatio","Op_profitmargin","SFL","EPSP","Medium","Big"]]
vif_data = pd.DataFrame()
vif_data["feature"] = x.columns

vif_data["VIF"] = [variance_inflation_factor(x.values,i)for i in range (len(x.columns))]
vif_data

,feature,VIF
0,B_MRatio,1.032862
1,Op_profitmargin,1.009563
2,SFL,1.032826
3,EPSP,1.032933
4,Medium,1.000015
5,Big,1.000037


Multicollinearity refers to when multiple variables in a model are correlated. We can mesure this with a high variance inflation Factor (VIF). We can notice that most of the variables have a VIF smaller than 10, we none of the variables have Multicollinearity.

### **Drop leverage points and outliers**

When a point is an outlier and has high leverage is centain to say it will have an affect on the model, so we drop these points:

In [31]:
intersect = np.intersect1d(leverage_index, outliers_index)
intersect

array([  68,   93,  368,  390,  631, 1236, 1262, 1318, 1361, 1599, 2826],
      dtype=int64)

In [32]:
dataSTL = dataSTL.drop(intersect, axis=0)
dataSTL

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,r,F1_ret,EPSP,SAG,B_MRatio,SFL,LFL,size,Medium,Big
0,A,2022q1,1674000.0,764000.0,5.340000e+05,0.0,-37000.00000,20000.000,36000.0,1.032700e+07,...,0.045405,-0.213296,0.007126,0.000000,0.129778,0.000000,0.264356,small,0,0
1,AA,2022q1,3293000.0,2181000.0,2.130000e+05,125000.0,-70000.00000,25000.000,210000.0,1.598800e+07,...,1.022496,0.217886,0.028098,0.000000,0.374854,0.000063,0.108019,small,0,0
2,AAIC,2022q1,8470.0,4773.0,0.000000e+00,0.0,-4111.00000,0.000,2287.0,9.208830e+05,...,-0.152090,-0.222528,-0.022229,-0.998389,1.794935,0.000000,0.178003,small,0,0
3,AAL,2022q1,8899000.0,0.0,1.062200e+07,0.0,92000.00000,455000.000,-451000.0,6.740100e+07,...,-0.269713,-0.514447,-0.138008,0.000000,-0.754610,0.035341,0.526120,small,0,0
4,AAME,2022q1,51608.0,0.0,4.781200e+04,0.0,0.00000,0.000,954.0,3.750310e+05,...,-0.156671,-0.475675,0.044556,0.000000,1.982019,0.000000,0.089961,small,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3602,ZVIA,2022q1,38034.0,23413.0,2.327500e+04,8901.0,6669.00000,0.000,12.0,1.164800e+05,...,0.000000,0.000000,-0.035760,0.000000,0.307988,0.005220,0.004155,small,0,0
3603,ZVO,2022q1,61633.0,39829.0,2.903600e+04,0.0,-127.00000,0.000,78.0,1.487510e+05,...,-1.599512,-0.982014,-0.266288,0.000000,0.403424,0.000000,0.000000,small,0,0
3604,ZWS,2022q1,239600.0,137700.0,5.690000e+04,1100.0,1100.00000,4800.000,10000.0,1.118600e+06,...,0.416636,0.093173,0.006782,0.000000,0.043502,0.005006,0.483014,small,0,0
3605,ZY,2022q1,4791.0,12455.0,5.608200e+04,-130.0,-532.00000,7994.000,-26.0,6.181890e+05,...,0.000000,-3.482115,-0.241984,0.000000,1.154503,0.093572,0.293062,small,0,0


Then we repete the process of obtaing the leverage points and the outliers until we do not have an intersection. 
Then we can calculate our model.

In [33]:
import statsmodels.formula.api as sm
result = sm.ols(formula="F1_ret ~ B_MRatio + Op_profitmargin + SFL + EPSP + Medium + Big", data=dataSTL).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 F1_ret   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     268.6
Date:                Mon, 17 Oct 2022   Prob (F-statistic):          1.26e-284
Time:                        21:55:51   Log-Likelihood:                -3019.5
No. Observations:                3596   AIC:                             6053.
Df Residuals:                    3589   BIC:                             6096.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3484      0.013    -